In [1]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from IPython.core.display import display, HTML
import scipy.constants as sc
from scipy.optimize import fsolve
from ipywidgets import interact, widgets
%matplotlib notebook

import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as sc
plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': r"\usepackage{amsmath} \boldmath"
})
import os

print(os.getpid())
%cd ../

19403
/media/work/docs/codes/c++/QSVT_framework/simulations


In [2]:
import QSP.examples.inverse_XW1D.initialization as initialization
import QSP.examples.inverse_XW1D.sim as sim
import QSP.examples.inverse_XW1D.plotting as pl


ModuleNotFoundError: No module named 'QSP'

In [8]:
import pylib.measurement as mse
import pylib.mix as mix
def get_norm(AA):
    N = len(AA)
    
    max_root = 0
    for i in range(N):
        sr = np.sqrt(np.sum(np.abs(AA[i, :])**2))
        if sr > max_root:
            max_root = sr
    return max_root

In [12]:
# --- Time simulation of Non-Hermitian (real) ---
mix.reload_module(mix)

def get_a(v):
#     return 2 * np.arccos(np.sqrt(np.abs(v)))  # for general state-preparation oracle
    return 2 * np.arccos(v)  # for real matrix

name_file_start = "AA"
path_root = "./tests/dynamic_non_hermitian/"

a11 = 4.2; a12 = 2;
a22 = 5.2;
a33 = -5.3;
a44 = 4.3;

a21 = np.conjugate(a12+1)

AA = np.array([
    [a11, a12, 0, 0],
    [a21, a22, 0, 0],
    [0, 0, a33, 0],
    [0, 0, 0, a44]
])
d = 2

# --- create normalized matrix and find its eig. v ---
coef_norm = get_norm(AA) * d
AA_norm = AA / coef_norm
U, S, VH = np.linalg.svd(AA_norm)  # AA_norm = U S VH

# print("-- U matrix --")
# mix.print_matrix(AA)

# print("\n-- normalized matrix --")
mix.print_matrix(AA_norm)

print("\n-- singular values --")
mix.print_array(S, ff=[16,6,"e"])

print("\n-- U matrix --")
mix.print_matrix(U, ff=[16,6,"e"])

print("\n-- VH matrix --")
mix.print_matrix(VH, ff=[16,6,"e"])

# --- calculate angles for the oracle ---
print("\n--- Oracle angles ---")
print("aa {:0.6e}".format(get_a(AA_norm[0,0]*d)))
print("ab {:0.6e}".format(get_a(AA_norm[0,1]*d)))
print("ba {:0.6e}".format(get_a(AA_norm[1,0]*d)))
print("ac {:0.6e}".format(get_a(AA_norm[1,1]*d)))
print("ad {:0.6e}".format(get_a(AA_norm[2,2])))
print("aw {:0.6e}".format(get_a(AA_norm[3,3])))

# --- write down the initial state to the .init file ---
ii1 = 2 # id of eigv, starting from 0

N = len(AA)
xr = np.real(U[:,ii1])
xi = np.imag(U[:,ii1])
res_init_state = xr + 1j*xi
norm_state = np.sqrt(np.sum(np.abs(res_init_state)**2))
res_init_state = res_init_state / norm_state
xr = xr / norm_state
xi = xi / norm_state
mix.write_init_state(name_file_start, path_root, xr, xi)

# --- Calculate the resulting state ---
t = 17

res_VH = np.transpose(VH)[:,ii1]
res_h = np.cos(t * S[ii1]) * res_init_state + 1j*np.sin(t * S[ii1]) * res_VH

norm_res = np.sqrt(np.sum(np.abs(res_h)**2))
res_h = res_h/norm_res
print("\n\n--- normalized result")
mix.print_array(res_h, ff = [18, 3, "e"])

--- 0 ---
        0.350           0.167           0.000           0.000
--- 1 ---
        0.250           0.433           0.000           0.000
--- 2 ---
        0.000           0.000          -0.441           0.000
--- 3 ---
        0.000           0.000           0.000           0.358

-- singular values --
    6.059993e-01     4.414215e-01     3.581344e-01     1.813169e-01 

-- U matrix --
--- 0 ---
   -5.921370e-01       0.000000e+00       0.000000e+00      -8.058373e-01
--- 1 ---
   -8.058373e-01       0.000000e+00       0.000000e+00       5.921370e-01
--- 2 ---
    0.000000e+00       1.000000e+00       0.000000e+00       0.000000e+00
--- 3 ---
    0.000000e+00       0.000000e+00       1.000000e+00       0.000000e+00

-- VH matrix --
--- 0 ---
   -6.740608e-01      -7.386758e-01      -0.000000e+00      -0.000000e+00
--- 1 ---
   -0.000000e+00      -0.000000e+00      -1.000000e+00      -0.000000e+00
--- 2 ---
    0.000000e+00       0.000000e+00       0.000000e+00       1.000000e+00

In [14]:
# --- Read the quantum state ---
mix.reload_module(mix)
mix.reload_module(mse)

oor = mse.MeasDyn__();
oor.pname_ = name_file_start
oor.path_  = path_root

oor.open()
oor.read_output_states()

print("--- Resulting relative amplitudes ---")
oor.print_rel_state(-1)

Reading a file ./tests/dynamic_non_hermitian//AA_OUTPUT.hdf5...
Name of the simulation is AA
Simulation has been performed  03-14-2022 23:23:45
--- Resulting relative amplitudes ---
Registers: 
{'qb': 2, 'a': 1, 'c': 2, 'r': 2}

                   -9.811e-01+1.937e-01j   |00>|0>|00>|11>


In [4]:
# --- For Hamiltonian simulation of the matrix AA0 ---
mix.reload_module(mix)

name_file_start = "AA"
path_root = "./QSP/examples/test_inverse/dynamic_hermitian/"

ii = 3 # id of eigv, starting from 0

eigenvectors_matrix = np.identity(4);
eigenvalues_all = [0.1020, 0.1020, 0.1020, 1.000]

N = 4
xr = np.real(eigenvectors_matrix[:,ii])
xi = np.imag(eigenvectors_matrix[:,ii])
norm_coef = 1

mix.write_init_state(name_file_start, path_root, xr, xi, norm_coef)

print("-- chosen eigenvector ---")
res_init_state = xr + 1j*xi
mix.print_array(res_init_state)

print("\n-- resulting exp(-i*t*lambda)|lambda> ---")
t = 17.82
# res_h = np.cos(t * eigenvalues_all[ii]) * res_init_state
# res_h = 1j* np.sin(t * eigenvalues_all[ii]) * res_init_state
res_h = np.exp(-1j * t * eigenvalues_all[ii]) * res_init_state
mix.print_array(res_h, ff = [18, 3, "e"])

writing the initial state of AA
-- chosen eigenvector ---
 0.000+0.000j  0.000+0.000j  0.000+0.000j  1.000+0.000j 

-- resulting exp(-i*t*lambda)|lambda> ---
0.000e+00+0.000e+00j 0.000e+00+0.000e+00j 0.000e+00+0.000e+00j 5.152e-01+8.571e-01j 


In [32]:
# --- Time simulation of AA2 ---
mix.reload_module(mix)

def get_a(v):
#     return 2 * np.arccos(np.sqrt(np.abs(v)))  # for general state-preparation oracle
    return 2 * np.arccos(v)  # for real matrix

name_file_start = "AA"
path_root = "./QSP/examples/test_inverse/dynamic_non_hermitian/"

ii1 = 0 # id of eigv, starting from 0
ii2 = 0 # id of eigv, starting from 0

a11 = 4.2; a12 = 2;
a22 = 5.2;
a33 = -5.3;
a44 = 4.3;

a21 = np.conjugate(a12)

AA = np.array([
    [a11, a12, 0, 0],
    [a21, a22, 0, 0],
    [0, 0, a33, 0],
    [0, 0, 0, a44]
])
d = 2

# --- create normalized matrix and find its eig. v ---
coef_norm = get_norm(AA) * d
AA_norm = AA / coef_norm
eigenvalues_all, eigenvectors_matrix = np.linalg.eig(AA_norm)

# print("-- matrix --")
# mix.print_matrix(AA)

# # print("\n-- normalized matrix --")
# mix.print_matrix(AA_norm)

print("\n-- eigenvalues --")
mix.print_array(eigenvalues_all, ff=[16,6,"e"])

print("\n-- eigenvectors --")
mix.print_matrix(eigenvectors_matrix, ff=[16,6,"e"])

# --- calculate angles for the oracle ---
print("\n--- Oracle angles ---")
print("aa {:0.6e}".format(get_a(AA_norm[0,0]*d)))
print("ab {:0.6e}".format(get_a(AA_norm[0,1]*d)))
print("ba {:0.6e}".format(get_a(AA_norm[1,0]*d)))
print("ac {:0.6e}".format(get_a(AA_norm[1,1]*d)))
print("ad {:0.6e}".format(get_a(AA_norm[2,2])))
print("aw {:0.6e}".format(get_a(AA_norm[3,3])))


# --- write down the initial state to the .init file ---
print()
N = len(AA)
xr = np.real(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
xi = np.imag(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
res_init_state = xr + 1j*xi
norm_state = np.sqrt(np.sum(np.abs(res_init_state)**2))
res_init_state = res_init_state / norm_state
xr = xr / norm_state
xi = xi / norm_state
mix.write_init_state(name_file_start, path_root, xr, xi, 1)

t = 17

res_h = np.exp(-1j * t * eigenvalues_all[ii1]) * res_init_state
norm_res = np.sqrt(np.sum(np.abs(res_h)**2))
res_h = res_h/norm_res
print("\n--- normalized result")
mix.print_array(res_h, ff = [18, 3, "e"])


-- eigenvalues --
    2.367868e-01     6.068140e-01    -4.756473e-01     3.859025e-01 

-- eigenvectors --
--- 0 ---
   -7.882054e-01      -6.154122e-01       0.000000e+00       0.000000e+00
--- 1 ---
    6.154122e-01      -7.882054e-01       0.000000e+00       0.000000e+00
--- 2 ---
    0.000000e+00       0.000000e+00       1.000000e+00       0.000000e+00
--- 3 ---
    0.000000e+00       0.000000e+00       0.000000e+00       1.000000e+00

--- Oracle angles ---
aa 1.433770e+00
ab 2.407245e+00
ba 2.407245e+00
ac 7.343477e-01
ad 4.132992e+00
aw 2.349221e+00

writing the initial state of AA

--- normalized result
4.999e-01-6.094e-01j -3.903e-01+4.758e-01j 0.000e+00+0.000e+00j 0.000e+00+0.000e+00j 


In [29]:
# --- Read the quantum state ---
mix.reload_module(mix)
mix.reload_module(mse)

oor = mse.MeasDyn__();
oor.pname_ = name_file_start
oor.path_  = path_root

oor.open()
oor.read_output_states()

print("--- Resulting relative amplitudes ---")
oor.print_rel_state(-1)

Reading a file ./QSP/examples/test_inverse/dynamic_non_hermitian//AA_OUTPUT.hdf5...
Name of the simulation is AA
Simulation has been performed  02-16-2022 19:12:02
--- Resulting relative amplitudes ---
Registers: 
{'qb': 2, 'a': 1, 'c': 2, 'r': 2}

                   -7.380e-01-1.215e-01j   |00>|0>|00>|00>
                    6.622e-01-4.640e-02j   |00>|0>|00>|01>


--- 0 ---
        0.350           0.167           0.000           0.000
--- 1 ---
        0.250           0.433           0.000           0.000
--- 2 ---
        0.000           0.000          -0.441           0.000
--- 3 ---
        0.000           0.000           0.000           0.358

-- singular values --
    6.059993e-01     4.414215e-01     3.581344e-01     1.813169e-01 

-- U matrix --
--- 0 ---
   -5.921370e-01       0.000000e+00       0.000000e+00      -8.058373e-01
--- 1 ---
   -8.058373e-01       0.000000e+00       0.000000e+00       5.921370e-01
--- 2 ---
    0.000000e+00       1.000000e+00       0.000000e+00       0.000000e+00
--- 3 ---
    0.000000e+00       0.000000e+00       1.000000e+00       0.000000e+00

-- VH matrix --
--- 0 ---
   -6.740608e-01      -7.386758e-01      -0.000000e+00      -0.000000e+00
--- 1 ---
   -0.000000e+00      -0.000000e+00      -1.000000e+00      -0.000000e+00
--- 2 ---
    0.000000e+00       0.000000e+00       0.000000e+00       1.000000e+00

Reading a file ./QSP/examples/test_inverse/dynamic_non_hermitian//AA_OUTPUT.hdf5...
Name of the simulation is AA
Simulation has been performed  02-22-2022 21:13:41
--- Resulting relative amplitudes ---
Registers: 
{'qb': 2, 'a': 1, 'c': 2, 'r': 2}

                   -5.921e-01+6.045e-16j   |00>|0>|00>|00>
                   -8.058e-01+4.670e-16j   |00>|0>|00>|01>


In [27]:
# --- Time evolution of AA7 (non-hermitian complex) ---

mix.reload_module(mix)

def get_a(v):
    return 2 * np.arccos(v)  # for real elements

def get_a_img(v):
    return 2 * np.arccos(np.abs(v))  # for real imaginary elements

name_file_start = "AA"
path_root = "./QSP/examples/test_inverse/dynamic_non_hermitian_complex/"

a11 = 4.2; 
a12 = 0.3 + 1j*1.0;
a21 = 2 + 1j*3.0;
a22 = 5.2;
a33 = 5.3;
a44 = 4.3;


# a11 = 4.2; 
# a12 = 0.3;
# a21 = 2;
# a22 = 5.2;
# a33 = 5.3;
# a44 = 4.3;


AA = np.array([
    [a11, a12, 0, 0],
    [a21, a22, 0, 0],
    [0, 0, a33, 0],
    [0, 0, 0, a44]
])
d = 2

# --- create normalized matrix and find its eig. v ---
coef_norm = get_norm(AA) * d
AA_norm = AA / coef_norm
U, S, VH = np.linalg.svd(AA_norm)  # AA_norm = U S VH

# print("-- U matrix --")
# mix.print_matrix(AA)

print("\n-- normalized matrix --")
mix.print_matrix(AA_norm)

print("\n-- singular values --")
mix.print_array(S, ff=[16,3,"e"])

print("\n-- U matrix --")
mix.print_matrix(U, ff=[16,3,"e"])

print("\n-- VH matrix --")
mix.print_matrix(VH, ff=[16,3,"e"])

# --- calculate angles for the oracle ---
print("\n--- Oracle angles ---")
print("aa {:20.6e}".format(get_a(AA_norm[0,0]*d)))

ww = cmath.polar(AA_norm[0,1]*d)
ab1 = -2.*ww[1]
ab2 = 2.*np.arccos(ww[0])
print("ab1 {:20.6e}".format(ab1)) # phase of the complex number (minus because of the Rz format)
print("ab2 {:20.6e}".format(ab2))  # magnitude of the complex number

ww = cmath.polar(AA_norm[1,0]*d)
ba1 = -2.*ww[1]
ba2 = 2.*np.arccos(ww[0])
print("ba1 {:20.6e}".format(ba1)) # phase of the complex number
print("ba2 {:20.6e}".format(ba2))  # magnitude of the complex number

print("ac {:20.6e}".format(get_a(AA_norm[1,1]*d)))
print("ad {:20.6e}".format(get_a(AA_norm[2,2])))
print("aw {:20.6e}".format(get_a(AA_norm[3,3])))


# --- Calculate exp ---
ii1 = 0

N = len(AA)
# xr = np.real(U[:,ii1])
# xi = np.imag(U[:,ii1])
xr = np.real(np.transpose(VH)[:,ii1])
xi = np.imag(np.transpose(VH)[:,ii1])
res_init_state = xr + 1j*xi
norm_state = np.sqrt(np.sum(np.abs(res_init_state)**2))
res_init_state = res_init_state / norm_state
xr = xr / norm_state
xi = xi / norm_state
mix.write_init_state(name_file_start, path_root, xr, xi, 1)


# --- Calculate the resulting state ---
t = 17

res_VH = np.conj(np.transpose(VH)[:,ii1]) 
# res_h = np.cos(t * S[ii1]) * res_init_state - 1j*np.sin(t * S[ii1]) * res_VH
res_h = np.cos(t * S[ii1]) * res_init_state

norm_res = np.sqrt(np.sum(np.abs(res_h)**2))
res_h = res_h/norm_res
print("\n\n--- normalized result")
mix.print_array(res_h, ff = [18, 3, "e"])


-- normalized matrix --
--- 0 ---
 0.332+0.000j    0.024+0.079j    0.000+0.000j    0.000+0.000j
--- 1 ---
 0.158+0.237j    0.411+0.000j    0.000+0.000j    0.000+0.000j
--- 2 ---
 0.000+0.000j    0.000+0.000j    0.419+0.000j    0.000+0.000j
--- 3 ---
 0.000+0.000j    0.000+0.000j    0.000+0.000j    0.340+0.000j

-- singular values --
       5.344e-01        4.188e-01        3.398e-01        2.852e-01 

-- U matrix --
--- 0 ---
-4.070e-01-9.290e-02j   0.000e+00+0.000e+00j   0.000e+00+0.000e+00j   8.890e-01-1.880e-01j
--- 1 ---
-5.905e-01-6.907e-01j   0.000e+00+0.000e+00j   0.000e+00+0.000e+00j   -3.793e-01-1.743e-01j
--- 2 ---
0.000e+00+0.000e+00j   1.000e+00+0.000e+00j   0.000e+00+0.000e+00j   0.000e+00+0.000e+00j
--- 3 ---
0.000e+00+0.000e+00j   0.000e+00+0.000e+00j   1.000e+00+0.000e+00j   0.000e+00+0.000e+00j

-- VH matrix --
--- 0 ---
-7.337e-01+0.000e+00j   -4.858e-01+4.750e-01j   -0.000e+00+0.000e+00j   -0.000e+00+0.000e+00j
--- 1 ---
0.000e+00+0.000e+00j   0.000e+00+0.000e+00j  

In [25]:
4.070e-01/9.290e-02

4.381054897739505

In [26]:
2.342e-01/5.854e-02

4.000683293474547

In [11]:
a1 = -2.327e-02  -1.879e-01j
a2 = -3.010e-01  -2.263e-01j

a1 = 

abs_a = np.sqrt(np.sum(np.abs([a1, a2])**2))
print("{:20.3e}".format(a1/abs_a))

-5.521e-02-4.458e-01j


In [28]:
# --- Read the quantum state ---
mix.reload_module(mix)
mix.reload_module(mse)

oor = mse.MeasDyn__();
oor.pname_ = name_file_start
oor.path_  = path_root

oor.open()
oor.read_output_states()

print("\n--- Resulting relative amplitudes ---")
oor.print_rel_state(-1)

Reading a file ./QSP/examples/test_inverse/dynamic_non_hermitian_complex//AA_OUTPUT.hdf5...
Name of the simulation is AA
Simulation has been performed  02-22-2022 21:21:22

--- Resulting relative amplitudes ---
Registers: 
{'qb': 2, 'a': 1, 'c': 2, 'r': 2}

                   -3.602e-01+4.304e-02j   |00>|0>|00>|00>
                   -6.980e-01+6.174e-01j   |00>|0>|00>|01>


In [20]:
# --- To inverse the matrix AA0 ---
mix.reload_module(mix)

name_file_start = "AA0"
path_root = "./QSP/examples/test_inverse/inverse_AA0/"

ii1 = 2 # id of eigv, starting from 0
ii2 = 3 # id of eigv, starting from 0

eigenvectors_matrix = np.identity(4);
# eigenvalues_all = [0.1020, 0.1020, 0.1020, 1.000]
eigenvalues_all = [0.5020, 0.5020, 0.5020, 1.000]
# eigenvalues_all = [0.3020, 0.3020, 0.3020, 1.000]

angles_oracle = 2*np.arccos(eigenvalues_all)

N = 4
xr = np.real(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
xi = np.imag(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
res_init_state = xr + 1j*xi
norm_state = np.sqrt(np.sum(np.abs(res_init_state)**2))
res_init_state = res_init_state / norm_state
xr = xr / norm_state
xi = xi / norm_state
mix.write_init_state(name_file_start, path_root, xr, xi, 1)

print("-- angles for oracle --")
mix.print_array(angles_oracle, ff = [16, 6, "e"])

print("-- init. state ---")
mix.print_array(res_init_state)

print("-- lambda_1 / lambda_2 ---")
print("{:0.3e}".format(eigenvalues_all[ii1]/eigenvalues_all[ii2]))

print("-- lambda_2 / lambda_1 ---")
print("{:0.3e}".format(eigenvalues_all[ii2]/eigenvalues_all[ii1]))

writing an initial state of AA0
-- angles for oracle --
    2.937237e+00     2.937237e+00     2.937237e+00     0.000000e+00 
-- init. state ---
 0.000+0.000j  0.000+0.000j  0.707+0.000j  0.707+0.000j 
-- lambda_1 / lambda_2 ---
1.020e-01
-- lambda_2 / lambda_1 ---
9.804e+00


In [14]:
# --- To inverse the matrix AA1 ---
mix.reload_module(mix)

name_file_start = "AA1"
path_root = "./QSP/examples/test_inverse/inverse-AA1/"

ii1 = 3 # id of eigv, starting from 0
ii2 = 3 # id of eigv, starting from 0

# for ad = 5.3
eigenvectors_matrix = np.array([
    [-0.7882,         0,         0,    0.6154],
    [ 0.6154,         0,         0,    0.7882],
    [      0,         0,    1.0000,         0],
    [      0,    1.0000,         0,         0]
])
eigenvalues_all = [0.2368, 0.3859, 0.4756, 0.6068]

N = 4
xr = np.real(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
xi = np.imag(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
res_init_state = xr + 1j*xi
norm_state = np.sqrt(np.sum(np.abs(res_init_state)**2))
res_init_state = res_init_state / norm_state
xr = xr / norm_state
xi = xi / norm_state
mix.write_init_state(name_file_start, path_root, xr, xi, 1)

print("-- init. state ---", end = "")
mix.print_array(res_init_state)

print("-- lambda_1 / lambda_2 ---")
print("{:0.3e}".format(eigenvalues_all[ii1]/eigenvalues_all[ii2]))

print("-- lambda_2 / lambda_1 ---")
print("{:0.3e}".format(eigenvalues_all[ii2]/eigenvalues_all[ii1]))

writing an initial state of AA1
-- init. state --- 0.615+0.000j  0.788+0.000j  0.000+0.000j  0.000+0.000j 
-- lambda_1 / lambda_2 ---
1.000e+00
-- lambda_2 / lambda_1 ---
1.000e+00


In [29]:
# --- To inverse the matrix AA2 ---
mix.reload_module(mix)

def get_a(v):
#     return 2 * np.arccos(np.sqrt(np.abs(v)))  # for general state-preparation oracle
    return 2 * np.arccos(v)  # for real matrix

name_file_start = "AA2"
path_root = "./QSP/examples/test_inverse/inverse_AA2/"

ii1 = 0 # id of eigv, starting from 0
ii2 = 2 # id of eigv, starting from 0

a11 = 4.2; a12 = 2;
a22 = 5.2;
a33 = -5.3;
a44 = 4.3;

a21 = np.conjugate(a12)

AA = np.array([
    [a11, a12, 0, 0],
    [a21, a22, 0, 0],
    [0, 0, a33, 0],
    [0, 0, 0, a44]
])
d = 2

# --- create normalized matrix and find its eig. v ---
coef_norm = get_norm(AA) * d
AA_norm = AA / coef_norm
eigenvalues_all, eigenvectors_matrix = np.linalg.eig(AA_norm)

print("-- matrix --")
mix.print_matrix(AA)

print("\n-- normalized matrix --")
mix.print_matrix(AA_norm)

print("\n-- eigenvalues --")
mix.print_array(eigenvalues_all, ff=[16,6,"e"])

print("\n-- eigenvectors --")
mix.print_matrix(eigenvectors_matrix, ff=[16,6,"e"])

# --- calculate angles for the oracle ---
print("\n--- Oracle angles ---")
print("aa: {:0.6e}".format(get_a(AA_norm[0,0]*d)))
print("ab: {:0.6e}".format(get_a(AA_norm[0,1]*d)))
print("ba: {:0.6e}".format(get_a(AA_norm[1,0]*d)))
print("ac: {:0.6e}".format(get_a(AA_norm[1,1]*d)))
print("ad: {:0.6e}".format(get_a(AA_norm[2,2])))
print("aw: {:0.6e}".format(get_a(AA_norm[3,3])))


# # --- write down the initial state to the .init file ---
# print()
# N = len(AA)
# xr = np.real(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
# xi = np.imag(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
# res_init_state = xr + 1j*xi
# norm_state = np.sqrt(np.sum(np.abs(res_init_state)**2))
# res_init_state = res_init_state / norm_state
# xr = xr / norm_state
# xi = xi / norm_state
# mix.write_init_state(name_file_start, path_root, xr, xi, 1)

# print("\n-- init. state ---")
# mix.print_array(res_init_state)

# print("\n-- lambda_1 / lambda_2 ---")
# print("{:0.3e}".format(eigenvalues_all[ii1]/eigenvalues_all[ii2]))

# print("\n-- lambda_2 / lambda_1 ---")
# print("{:0.3e}".format(eigenvalues_all[ii2]/eigenvalues_all[ii1]))

# --- Consider a system of linear equations ---
# --- write down the initial state to the .init file ---
b_r = np.array([1.0, 0.0, 0.0, 0.0])
b_i = np.array([0.0, 0.0, 0.0, 0.0])
b = b_r + 1j * b_i
coef_norm = np.sqrt(np.sum(np.abs(b)**2))

b = b/coef_norm
b_r = b_r/coef_norm
b_i = b_i/coef_norm
A_norm_inv = np.linalg.inv(AA_norm)
x = np.dot(A_norm_inv, b)

mix.write_init_state(name_file_start, path_root, b_r, b_i, 1)

print("\n-- init. state ---")
mix.print_array(b)

print("\n-- resulting state ---")
mix.print_array(x)

-- matrix --
--- 0 ---
        4.200           2.000           0.000           0.000
--- 1 ---
        2.000           5.200           0.000           0.000
--- 2 ---
        0.000           0.000          -5.300           0.000
--- 3 ---
        0.000           0.000           0.000           4.300

-- normalized matrix --
--- 0 ---
        0.377           0.179           0.000           0.000
--- 1 ---
        0.179           0.467           0.000           0.000
--- 2 ---
        0.000           0.000          -0.476           0.000
--- 3 ---
        0.000           0.000           0.000           0.386

-- eigenvalues --
    2.367868e-01     6.068140e-01    -4.756473e-01     3.859025e-01 

-- eigenvectors --
--- 0 ---
   -7.882054e-01      -6.154122e-01       0.000000e+00       0.000000e+00
--- 1 ---
    6.154122e-01      -7.882054e-01       0.000000e+00       0.000000e+00
--- 2 ---
    0.000000e+00       0.000000e+00       1.000000e+00       0.000000e+00
--- 3 ---
    0.000000e+00

In [32]:
print( "theory: state 0 / state 1: \t", -0.557/0.435  )
print( "sim: state 0 / state 1: \t", -2.942e-02 / 2.297e-02 )

theory: state 0 / state 1: 	 -1.2804597701149427
sim: state 0 / state 1: 	 -1.2808010448410971


In [30]:
print( "theory: state 0 / state 2: \t", (-0.557/2.367868e-01)/(0.707/-4.756473e-01)  )
print( "sim: state 0 / state 2: \t", -2.942e-02 / -1.858e-02 )

theory: state 0 / state 2: 	 1.5825714577790666
sim: state 0 / state 2: 	 1.5834230355220669


In [31]:
print( "theory: state 1 / state 2: \t", (0.435/2.367868e-01)/(0.707/-4.756473e-01)  )
print( "sim: state 1 / state 2: \t", 2.297e-02 / -1.858e-02 )

theory: state 1 / state 2: 	 -1.2359400074217124
sim: state 1 / state 2: 	 -1.236275565123789


In [29]:
# --- To inverse the matrix AA3 ---
mix.reload_module(mix)

def get_a(v):
    return 2 * np.arccos(v)  # for real elements

def get_a_img(v):
    return 2 * np.arccos(np.abs(v))  # for real imaginary elements

name_file_start = "AA3"
path_root = "./QSP/examples/test_inverse/inverse_AA3/"

ii1 = 0 # id of eigv, starting from 0
ii2 = 1 # id of eigv, starting from 0

a11 = 4.2; 
a12 = 1j*1.0;
a22 = 5.2;
a33 = 5.3;
a44 = 4.3;

# a11 = 0; 
# a12 = 1j*1.0;
# a22 = 0;
# a33 = 1;
# a44 = 1;

a21 = np.conjugate(a12)

AA = np.array([
    [a11, a12, 0, 0],
    [a21, a22, 0, 0],
    [0, 0, a33, 0],
    [0, 0, 0, a44]
])
d = 2

# --- create normalized matrix and find its eig. v ---
coef_norm = get_norm(AA) * d
AA_norm = AA / coef_norm
eigenvalues_all, eigenvectors_matrix = np.linalg.eig(AA_norm)

print("-- matrix --")
mix.print_matrix(AA)

print("\n-- normalized matrix --")
mix.print_matrix(AA_norm)

print("\n-- eigenvalues --")
mix.print_array(eigenvalues_all, ff=[16,3,"e"])

print("\n-- eigenvectors --")
mix.print_matrix(eigenvectors_matrix, ff=[16,3,"e"])

# --- calculate angles for the oracle ---
print("\n--- Oracle angles ---")
print("aa: {:20.6e}".format(get_a(AA_norm[0,0]*d)))
print("ab: {:20.6e}".format(get_a_img(AA_norm[0,1]*d)))
print("ba: {:20.6e}".format(get_a_img(AA_norm[1,0]*d)))
print("ac: {:20.6e}".format(get_a(AA_norm[1,1]*d)))
print("ad: {:20.6e}".format(get_a(AA_norm[2,2])))
print("aw: {:20.6e}".format(get_a(AA_norm[3,3])))


# # --- write down the initial state to the .init file ---
# print()
# N = len(AA)
# xr = np.real(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
# xi = np.imag(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
# res_init_state = xr + 1j*xi
# norm_state = np.sqrt(np.sum(np.abs(res_init_state)**2))
# res_init_state = res_init_state / norm_state
# xr = xr / norm_state
# xi = xi / norm_state
# mix.write_init_state(name_file_start, path_root, xr, xi, 1)

# print("\n-- init. state ---")
# mix.print_array(res_init_state)

# print("\n-- lambda_1 / lambda_2 ---")
# print("{:20.3e}".format(eigenvalues_all[ii1]/eigenvalues_all[ii2]))

# print("\n-- lambda_2 / lambda_1 ---")
# print("{:20.3e}".format(eigenvalues_all[ii2]/eigenvalues_all[ii1]))

# --- Consider a system of linear equations ---
# --- write down the initial state to the .init file ---
b_r = np.array([1.0, 0.0, 0.0, 0.0])
b_i = np.array([0.0, -1.0, 0.0, 0.0])
b = b_r + 1j * b_i
coef_norm = np.sqrt(np.sum(np.abs(b)**2))

b = b/coef_norm
b_r = b_r/coef_norm
b_i = b_i/coef_norm
A_norm_inv = np.linalg.inv(AA_norm)
x = np.dot(A_norm_inv, b)

mix.write_init_state(name_file_start, path_root, b_r, b_i, 1)

print("\n-- init. state ---")
mix.print_array(b)

print("\n-- resulting state ---")
mix.print_array(x)

-- matrix --
--- 0 ---
 4.200+0.000j    0.000+1.000j    0.000+0.000j    0.000+0.000j
--- 1 ---
 0.000-1.000j    5.200+0.000j    0.000+0.000j    0.000+0.000j
--- 2 ---
 0.000+0.000j    0.000+0.000j    5.300+0.000j    0.000+0.000j
--- 3 ---
 0.000+0.000j    0.000+0.000j    0.000+0.000j    4.300+0.000j

-- normalized matrix --
--- 0 ---
 0.396+0.000j    0.000+0.094j    0.000+0.000j    0.000+0.000j
--- 1 ---
 0.000-0.094j    0.491+0.000j    0.000+0.000j    0.000+0.000j
--- 2 ---
 0.000+0.000j    0.000+0.000j    0.500+0.000j    0.000+0.000j
--- 3 ---
 0.000+0.000j    0.000+0.000j    0.000+0.000j    0.406+0.000j

-- eigenvalues --
3.379e-01+0.000e+00j 5.489e-01+0.000e+00j 5.000e-01+0.000e+00j 4.057e-01+0.000e+00j 

-- eigenvectors --
--- 0 ---
8.507e-01+0.000e+00j   0.000e+00+5.257e-01j   0.000e+00+0.000e+00j   0.000e+00+0.000e+00j
--- 1 ---
-0.000e+00+5.257e-01j   8.507e-01+0.000e+00j   0.000e+00+0.000e+00j   0.000e+00+0.000e+00j
--- 2 ---
-0.000e+00-0.000e+00j   0.000e+00-0.000e+00j   1.00

In [26]:
x[0]/x[1]

(-0+1j)

In [17]:
1.768e-02/-1.768e-02j

(-0+1j)

In [20]:
1.768e-02j/-1.768e-02

(-0-1j)

In [28]:
-1.768e-02/(-1.768e-02j)

(-0-1j)

In [7]:
1.888e-02/-1.439e-02j

(-0+1.3120222376650452j)

In [10]:
 -1.888e-02j/-1.439e-02

(-0+1.3120222376650452j)

In [56]:
6.624e-01/-1.848e-01j

(-0+3.5844155844155847j)

In [54]:
3.306e-02/6.358e-03j

-5.199748348537276j

In [55]:
4.582e-01/-6.410e-01j

(-0+0.7148205928237129j)

In [127]:
# --- To inverse the matrix AA5 ---

mix.reload_module(mix)

def get_a(v):
    return 2 * np.arccos(v)  # for real elements

def get_a_img(v):
    return 2 * np.arccos(np.abs(v))  # for real imaginary elements

name_file_start = "AA5"
path_root = "./QSP/examples/test_inverse/inverse_AA5/"

ii1 = 0 # id of eigv, starting from 0
ii2 = 1 # id of eigv, starting from 0

a11 = 4.2; 
a12 = 0.3 + 1j*1.0;
a22 = 5.2;
a33 = -5.3;
a44 = 4.3;

a21 = np.conjugate(a12)

AA = np.array([
    [a11, a12, 0, 0],
    [a21, a22, 0, 0],
    [0, 0, a33, 0],
    [0, 0, 0, a44]
])
d = 2

# --- create normalized matrix and find its eig. v ---
coef_norm = get_norm(AA) * d
AA_norm = AA / coef_norm
eigenvalues_all, eigenvectors_matrix = np.linalg.eig(AA_norm)

print("-- matrix --")
mix.print_matrix(AA)

print("\n-- normalized matrix --")
mix.print_matrix(AA_norm)



print("\n-- eigenvalues --")
mix.print_array(eigenvalues_all, ff=[16,3,"e"])

print("\n-- eigenvectors --")
mix.print_matrix(eigenvectors_matrix, ff=[16,3,"e"])

# --- calculate angles for the oracle ---
print("\n--- Oracle angles ---")
print("aa: {:20.6e}".format(get_a(AA_norm[0,0]*d)))

ww = cmath.polar(AA_norm[0,1]*d)
ab1 = -2.*ww[1]
ab2 = 2.*np.arccos(ww[0])
print("ab1: {:20.6e}".format(ab1)) # phase of the complex number (minus because of Rz format)
print("ab2: {:20.6e}".format(ab2))  # magnitude of the complex number

ww = cmath.polar(AA_norm[1,0]*d)
ba1 = -2.*ww[1]
ba2 = 2.*np.arccos(ww[0])
print("ba1: {:20.6e}".format(ba1)) # phase of the complex number
print("ba2: {:20.6e}".format(ba2))  # magnitude of the complex number

print("ac: {:20.6e}".format(get_a(AA_norm[1,1]*d)))
print("ad: {:20.6e}".format(get_a(AA_norm[2,2])))
print("aw: {:20.6e}".format(get_a(AA_norm[3,3])))

# # --- write down the initial state to the .init file ---
# print()
# N = len(AA)
# xr = np.real(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
# xi = np.imag(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
# res_init_state = xr + 1j*xi
# norm_state = np.sqrt(np.sum(np.abs(res_init_state)**2))
# res_init_state = res_init_state / norm_state
# xr = xr / norm_state
# xi = xi / norm_state
# mix.write_init_state(name_file_start, path_root, xr, xi, 1)

# print("\n-- init. state ---")
# mix.print_array(res_init_state)

# print("\n-- lambda_1 / lambda_2 ---")
# print("{:20.3e}".format(eigenvalues_all[ii1]/eigenvalues_all[ii2]))

# print("\n-- lambda_2 / lambda_1 ---")
# print("{:20.3e}".format(eigenvalues_all[ii2]/eigenvalues_all[ii1]))


# --- Consider a system of linear equations ---
# --- write down the initial state to the .init file ---
b_r = np.array([0.0, 1.0, 0.0, 0.0])
b_i = np.array([1.0, 0.0, 0.0, 0.0])
b = b_r + 1j * b_i
coef_norm = np.sqrt(np.sum(np.abs(b)**2))

b = b/coef_norm
b_r = b_r/coef_norm
b_i = b_i/coef_norm
A_norm_inv = np.linalg.inv(AA_norm)
x = np.dot(A_norm_inv, b)

mix.write_init_state(name_file_start, path_root, b_r, b_i, 1)

print("\n-- init. state ---")
mix.print_array(b)

print("\n-- resulting state ---")
mix.print_array(x)

-- matrix --
--- 0 ---
 4.200+0.000j    0.300+1.000j    0.000+0.000j    0.000+0.000j
--- 1 ---
 0.300-1.000j    5.200+0.000j    0.000+0.000j    0.000+0.000j
--- 2 ---
 0.000+0.000j    0.000+0.000j   -5.300+0.000j    0.000+0.000j
--- 3 ---
 0.000+0.000j    0.000+0.000j    0.000+0.000j    4.300+0.000j

-- normalized matrix --
--- 0 ---
 0.396+0.000j    0.028+0.094j    0.000+0.000j    0.000+0.000j
--- 1 ---
 0.028-0.094j    0.490+0.000j    0.000+0.000j    0.000+0.000j
--- 2 ---
 0.000+0.000j    0.000+0.000j   -0.500+0.000j    0.000+0.000j
--- 3 ---
 0.000+0.000j    0.000+0.000j    0.000+0.000j    0.405+0.000j

-- eigenvalues --
3.340e-01+1.577e-17j 5.522e-01+3.974e-17j -4.996e-01+0.000e+00j 4.054e-01+0.000e+00j 

-- eigenvectors --
--- 0 ---
8.461e-01+0.000e+00j   1.531e-01+5.105e-01j   0.000e+00+0.000e+00j   0.000e+00+0.000e+00j
--- 1 ---
-1.531e-01+5.105e-01j   8.461e-01+0.000e+00j   0.000e+00+0.000e+00j   0.000e+00+0.000e+00j
--- 2 ---
-0.000e+00+0.000e+00j   0.000e+00+0.000e+00j   1.0

In [128]:
x[0]/x[1]

(-0.2149080348499516+1.2923523717328174j)

In [126]:
x[0]/x[1]

(-0.12605971249539255-1.1850350165868042j)

In [125]:
(1.356e-03j +  1.898e-02)/(-1.446e-02j  -1.356e-03)

(-0.21497448332894134+1.2924270124900386j)

In [7]:
# --- To inverse the matrix AA6 (non-hermitian) ---

mix.reload_module(mix)

def get_a(v):
    return 2 * np.arccos(v)  # for real elements

def get_a_img(v):
    return 2 * np.arccos(np.abs(v))  # for real imaginary elements

name_file_start = "AA6"
path_root = "./QSP/examples/test_inverse/inverse_AA6/"

ii1 = 0 # id of eigv, starting from 0
ii2 = 1 # id of eigv, starting from 0

a11 = 4.2; 
a12 = 0.3;
a22 = 5.2;
a33 = -5.3;
a44 = 4.3;

a21 = 1.0

AA = np.array([
    [a11, a12, 0, 0],
    [a21, a22, 0, 0],
    [0, 0, a33, 0],
    [0, 0, 0, a44]
])
d = 2

# --- create normalized matrix and find its eig. v ---
coef_norm = get_norm(AA) * d
AA_norm = AA / coef_norm
eigenvalues_all, eigenvectors_matrix = np.linalg.eig(AA_norm)

print("-- matrix --")
mix.print_matrix(AA)

print("\n-- normalized matrix --")
mix.print_matrix(AA_norm)

print("\n-- eigenvalues --")
mix.print_array(eigenvalues_all, ff=[16,3,"e"])

print("\n-- eigenvectors --")
mix.print_matrix(eigenvectors_matrix, ff=[16,3,"e"])

# --- calculate angles for the oracle ---
print("\n--- Oracle angles ---")
print("aa: {:20.6e}".format(get_a(AA_norm[0,0]*d)))
print("ab: {:20.6e}".format(get_a(AA_norm[0,1]*d))) # phase of the complex number (minus because of Rz format)
print("ba: {:20.6e}".format(get_a(AA_norm[1,0]*d))) # phase of the complex number
print("ac: {:20.6e}".format(get_a(AA_norm[1,1]*d)))
print("ad: {:20.6e}".format(get_a(AA_norm[2,2])))
print("aw: {:20.6e}".format(get_a(AA_norm[3,3])))

# # --- write down the initial state to the .init file ---
# print()
# N = len(AA)
# xr = np.real(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
# xi = np.imag(eigenvectors_matrix[:,ii1] + eigenvectors_matrix[:,ii2])
# res_init_state = xr + 1j*xi
# norm_state = np.sqrt(np.sum(np.abs(res_init_state)**2))
# res_init_state = res_init_state / norm_state
# xr = xr / norm_state
# xi = xi / norm_state
# mix.write_init_state(name_file_start, path_root, xr, xi, 1)

# print("\n-- init. state ---")
# mix.print_array(res_init_state)

# print("\n-- lambda_1 / lambda_2 ---")
# print("{:20.3e}".format(eigenvalues_all[ii1]/eigenvalues_all[ii2]))

# print("\n-- lambda_2 / lambda_1 ---")
# print("{:20.3e}".format(eigenvalues_all[ii2]/eigenvalues_all[ii1]))


# --- Consider a system of linear equations ---
# --- write down the initial state to the .init file ---
b_r = np.array([1.0, 1.0, 1.0, 1.0])
b_i = np.array([1.0, 1.0, 1.0, 1.0])
b = b_r + 1j * b_i
coef_norm = np.sqrt(np.sum(np.abs(b)**2))

b = b/coef_norm
b_r = b_r/coef_norm
b_i = b_i/coef_norm
A_norm_inv = np.linalg.inv(AA_norm)
x = np.dot(A_norm_inv, b)

mix.write_init_state(name_file_start, path_root, b_r, b_i, 1)

print("\n-- init. state ---")
mix.print_array(b)

print("\n-- resulting state ---")
mix.print_array(x)

-- matrix --
--- 0 ---
        4.200           0.300           0.000           0.000
--- 1 ---
        1.000           5.200           0.000           0.000
--- 2 ---
        0.000           0.000          -5.300           0.000
--- 3 ---
        0.000           0.000           0.000           4.300

-- normalized matrix --
--- 0 ---
        0.396           0.028           0.000           0.000
--- 1 ---
        0.094           0.491           0.000           0.000
--- 2 ---
        0.000           0.000          -0.500           0.000
--- 3 ---
        0.000           0.000           0.000           0.406

-- eigenvalues --
       3.734e-01        5.134e-01       -5.000e-01        4.057e-01 

-- eigenvectors --
--- 0 ---
      -7.788e-01         -2.349e-01          0.000e+00          0.000e+00
--- 1 ---
       6.273e-01         -9.720e-01          0.000e+00          0.000e+00
--- 2 ---
       0.000e+00          0.000e+00          1.000e+00          0.000e+00
--- 3 ---
       0.000e+00

In [8]:
x[0]/x[1]

(1.53125+0j)

In [11]:
(-1.066e-02j  + 1.066e-02) / (-6.960e-03j  + 6.960e-03)

(1.5316091954022988+0j)

In [12]:
x[0]/x[2]

(-1.2056638811513463-0j)

In [14]:
(-1.066e-02j  + 1.066e-02) / (8.840e-03j  -8.840e-03)

(-1.2058823529411764-0j)

In [15]:
x[0]/x[3]

(0.9781801299907148+0j)

In [16]:
(-1.066e-02j  + 1.066e-02) / (-1.090e-02j + 1.090e-02)

(0.9779816513761467+0j)

In [17]:
x[1]/x[2]

(-0.787372330547818-0j)

In [19]:
(-6.960e-03j+   6.960e-03) / (8.840e-03j  -8.840e-03)

(-0.7873303167420814-0j)

In [20]:
x[1]/x[3]

(0.638811513463324+0j)

In [22]:
(-6.960e-03j+   6.960e-03) / (-1.090e-02j   +1.090e-02)

(0.6385321100917432+0j)

In [23]:
x[2]/x[3]

(-0.8113207547169811+0j)

In [24]:
(8.840e-03j  -8.840e-03) / (-1.090e-02j  + 1.090e-02)

(-0.8110091743119267+0j)

In [20]:
# --- To inverse the matrix AA7 (non-hermitian complex) ---

mix.reload_module(mix)

def get_a(v):
    return 2 * np.arccos(v)  # for real elements

def get_a_img(v):
    return 2 * np.arccos(np.abs(v))  # for real imaginary elements

name_file_start = "AA7"
path_root = "./QSP/examples/test_inverse/inverse_AA7/"

a11 = 4.2; 
a12 = 0.3 + 1j*1.0;
a22 = 5.2;
a33 = 5.3;
a44 = 4.3;

a21 = 2 + 1j*3.0;

AA = np.array([
    [a11, a12, 0, 0],
    [a21, a22, 0, 0],
    [0, 0, a33, 0],
    [0, 0, 0, a44]
])
d = 2

# --- create normalized matrix and find its eig. v ---
coef_norm = get_norm(AA) * d
AA_norm = AA / coef_norm
eigenvalues_all, eigenvectors_matrix = np.linalg.eig(AA_norm)

print("-- matrix --")
mix.print_matrix(AA)

print("\n-- normalized matrix --")
mix.print_matrix(AA_norm)

print("\n-- eigenvalues --")
mix.print_array(eigenvalues_all, ff=[16,3,"e"])

print("\n-- eigenvectors --")
mix.print_matrix(eigenvectors_matrix, ff=[16,3,"e"])

# --- calculate angles for the oracle ---
print("\n--- Oracle angles ---")
print("aa: {:20.6e}".format(get_a(AA_norm[0,0]*d)))

ww = cmath.polar(AA_norm[0,1]*d)
ab1 = -2.*ww[1]
ab2 = 2.*np.arccos(ww[0])
print("ab1: {:20.6e}".format(ab1)) # phase of the complex number (minus because of Rz format)
print("ab2: {:20.6e}".format(ab2))  # magnitude of the complex number

ww = cmath.polar(AA_norm[1,0]*d)
ba1 = -2.*ww[1]
ba2 = 2.*np.arccos(ww[0])
print("ba1: {:20.6e}".format(ba1)) # phase of the complex number
print("ba2: {:20.6e}".format(ba2))  # magnitude of the complex number

print("ac: {:20.6e}".format(get_a(AA_norm[1,1]*d)))
print("ad: {:20.6e}".format(get_a(AA_norm[2,2])))
print("aw: {:20.6e}".format(get_a(AA_norm[3,3])))

# --- Consider a system of linear equations ---
# --- write down the initial state to the .init file ---
b_r = np.array([1.0, 1.0, 1.0, 1.0])
b_i = np.array([1.0, 1.0, 1.0, 1.0])
b = b_r + 1j * b_i
coef_norm = np.sqrt(np.sum(np.abs(b)**2))

b = b/coef_norm
b_r = b_r/coef_norm
b_i = b_i/coef_norm
A_norm_inv = np.linalg.inv(AA_norm)
x = np.dot(A_norm_inv, b)

mix.write_init_state(name_file_start, path_root, b_r, b_i, 1)

print("\n-- init. state ---")
mix.print_array(b)

print("\n-- resulting state ---")
mix.print_array(x)

-- matrix --
--- 0 ---
 4.200+0.000j    0.300+1.000j    0.000+0.000j    0.000+0.000j
--- 1 ---
 2.000+3.000j    5.200+0.000j    0.000+0.000j    0.000+0.000j
--- 2 ---
 0.000+0.000j    0.000+0.000j    5.300+0.000j    0.000+0.000j
--- 3 ---
 0.000+0.000j    0.000+0.000j    0.000+0.000j    4.300+0.000j

-- normalized matrix --
--- 0 ---
 0.332+0.000j    0.024+0.079j    0.000+0.000j    0.000+0.000j
--- 1 ---
 0.158+0.237j    0.411+0.000j    0.000+0.000j    0.000+0.000j
--- 2 ---
 0.000+0.000j    0.000+0.000j    0.419+0.000j    0.000+0.000j
--- 3 ---
 0.000+0.000j    0.000+0.000j    0.000+0.000j    0.340+0.000j

-- eigenvalues --
3.039e-01-1.341e-01j 4.389e-01+1.341e-01j 4.188e-01+0.000e+00j 3.398e-01+0.000e+00j 

-- eigenvectors --
--- 0 ---
-5.140e-01+4.409e-02j   4.021e-01+1.616e-01j   0.000e+00+0.000e+00j   0.000e+00+0.000e+00j
--- 1 ---
8.567e-01+0.000e+00j   9.012e-01+0.000e+00j   0.000e+00+0.000e+00j   0.000e+00+0.000e+00j
--- 2 ---
0.000e+00+0.000e+00j   -0.000e+00+0.000e+00j   1.00

In [22]:
x[0]/x[1]

(0.9956647398843929+0.9031791907514454j)

In [23]:
(-1.236e-02j + 1.048e-02) / (-1.205e-02j - 4.047e-04)

(0.9953941307408173+0.9031399174033866j)

In [24]:
x[0]/x[2]

(1.082040633060151-0.08919480875105457j)

In [25]:
(-1.236e-02j + 1.048e-02) / (-1.055e-02j + 1.055e-02)

(1.0824644549763032-0.0890995260663507j)

In [26]:
x[0]/x[3]

(0.8778820230488018-0.0723655995527424j)

In [27]:
(-1.236e-02j + 1.048e-02) / (-1.301e-02j  + 1.301e-02)

(0.8777863182167562-0.07225211375864718j)

In [28]:
x[1]/x[2]

(0.5516027514666411-0.5899485157073739j)

In [29]:
(-1.205e-02j  -4.047e-04) / (-1.055e-02j  + 1.055e-02) 

(0.5519099526066351-0.5902701421800948j)

In [30]:
x[1]/x[3]

(0.44752676062387864-0.47863747500786946j)

In [31]:
(-1.205e-02j -4.047e-04) / (-1.301e-02j  + 1.301e-02)

(0.4475518831667948-0.47865872405841653j)

In [32]:
x[2]/x[3]

(0.8113207547169812+0j)

In [33]:
(-1.055e-02j  + 1.055e-02) / (-1.301e-02j  + 1.301e-02)

(0.8109146810146042+0j)

In [14]:
x[0]/x[1]

(-0.18911174785100274+0.5308022922636103j)

In [13]:
(-2.837e-02j+2.010e-03) / (9.752e-03j  + 4.371e-03)

(-2.3455496589114455-1.2574238678324372j)

In [38]:
x[0]/x[1]

(-2.345588235294118-1.2573529411764708j)

In [37]:
(3.651e-03j   +1.894e-02) / (-3.385e-02j  -5.180e-03)

(-0.18905438420118315+0.5305966998474998j)

In [ ]:
# --- For Hamiltonian simulation of the matrix NH0 ---
mix.reload_module(mix)

def get_a(v):
    return 2 * np.arccos(v)  # for real matrix

name_file_start = "NH0"
path_root = "./QSP/examples/test_inverse/hsim_non_hermitian_tiny/"

ii = 0 # id of eigv, starting from 0

AA = np.array([
    [-1.j, 0.],
    [0.,  1.],
])
d = 1

# --- create normalized matrix and find its eig. v ---
coef_norm = get_norm(AA) * d
AA_norm = AA / coef_norm
eigenvalues_all, eigenvectors_matrix = np.linalg.eig(AA_norm)

print("-- matrix --")
mix.print_matrix(AA)

print("\n-- normalized matrix --")
mix.print_matrix(AA_norm)

print("\n-- eigenvalues --")
mix.print_array(eigenvalues_all, ff=[16,6,"e"])

print("\n-- eigenvectors --")
mix.print_matrix(eigenvectors_matrix, ff=[16,6,"e"])

# # --- calculate angles for the oracle ---
# print("\n--- Oracle angles ---")
# print("aa: {:0.6e}".format(get_a(AA_norm[0,0]*d)))
# print("ab: {:0.6e}".format(get_a(AA_norm[0,1]*d)))
# print("ba: {:0.6e}".format(get_a(AA_norm[1,0]*d)))
# print("ac: {:0.6e}".format(get_a(AA_norm[1,1]*d)))
# print("ad: {:0.6e}".format(get_a(AA_norm[2,2])))
# print("aw: {:0.6e}".format(get_a(AA_norm[3,3])))


# --- write down the initial state to the .init file ---
print()
N = len(AA)
xr = np.real(eigenvectors_matrix[:,ii])
xi = np.imag(eigenvectors_matrix[:,ii])
res_init_state = xr + 1j*xi
mix.write_init_state(name_file_start, path_root, xr, xi, 1)

print("\n-- init. state ---")
mix.print_array(res_init_state)

print("\n-- resulting exp(-i*t*lambda)|lambda> ---")
t = 10
res_h = np.exp(-1j * t * eigenvalues_all[ii]) * res_init_state
mix.print_array(res_h, ff = [18, 3, "e"])